### Imports

In [1]:
import torch
import warnings
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import mean_absolute_error

### Auglichem imports

In [2]:
from auglichem.crystal import (PerturbStructureTransformation,
                               RotationTransformation,
                               SwapAxesTransformation,
                               TranslateSitesTransformation,
                               SupercellTransformation,
)
from auglichem.crystal.data import CrystalDatasetWrapper
from auglichem.crystal.models import SchNet, GINet

### Set up dataset

In [3]:
# Create transformation
transforms = [
        PerturbStructureTransformation(distance=0.1, min_distance=0.01),
        RotationTransformation(axis=[0,0,1], angle=90),
        SwapAxesTransformation(),
        TranslateSitesTransformation(indices_to_move=[0], translation_vector=[1,0,0],
                                     vector_in_frac_coords=True),
        SupercellTransformation(scaling_matrix=[[1,0,0],[0,1,0],[0,0,1]]),
]

# Initialize dataset object
dataset = CrystalDatasetWrapper("lanthanides", batch_size=128,
                                valid_size=0.1, test_size=0.1)

# Get train/valid/test splits as loaders
train_loader, valid_loader, test_loader = dataset.get_data_loaders(transform=transforms)

Data found at: ./data_download/lanths


100%|█████████████████████████████████████████████████████████| 3332/3332 [00:00<00:00, 57410.67it/s]
/Users/clo/miniforge3/envs/auglichem/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


### Initialize model with task from data

In [4]:
# Get model
model = GINet() # Note: SchNet and GINet are interchangeable in use cases

# Uncomment the following line to use cuda
#model.cuda()

### Initialize traning loop

In [5]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

### Train the model

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for epoch in range(1):
        for bn, data in tqdm(enumerate(train_loader)):        
            optimizer.zero_grad()

            # Comment out the following line and uncomment the line after for cuda
            pred = model(data)
            #pred = model(data.cuda())
            
            loss = criterion(pred, data.y)

            loss.backward()
            optimizer.step()

157it [02:56,  1.12s/it]


### Test the model

In [7]:
def evaluate(model, test_loader, validation=False):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with torch.no_grad():
            model.eval()
            preds = torch.Tensor([])
            targets = torch.Tensor([])
            for data in test_loader:
                pred = model(data)
                #pred = model(data.cuda())
                preds = torch.cat((preds, pred.cpu()))
                targets = torch.cat((targets, data.y.cpu()))

            mae = mean_absolute_error(preds, targets)   
        
        set_str = "VALIDATION" if(validation) else "TEST"
        print("{0} MAE: {1:.3f}".format(set_str, mae))

In [8]:
evaluate(model, valid_loader, validation=True)
evaluate(model, test_loader)

VALIDATION MAE: 0.434
TEST MAE: 0.437


### Model saving/loading example

In [9]:
# Save model
torch.save(model.state_dict(), "./saved_models/example_gin")

In [10]:
# Instantiate new model and evaluate
model = GINet()

evaluate(model, valid_loader, validation=True)
evaluate(model, test_loader)

VALIDATION MAE: 1.084
TEST MAE: 1.082


In [11]:
# Load saved model and evaluate
model.load_state_dict(torch.load("./saved_models/example_gin"))
evaluate(model, valid_loader, validation=True)
evaluate(model, test_loader)

VALIDATION MAE: 0.434
TEST MAE: 0.437
